In [4]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import PandasTools, rdFingerprintGenerator, DataStructs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib

# Load SDF file and convert to DataFrame
sdf_file = "Structures_759_training.sdf"  
df = PandasTools.LoadSDF(sdf_file, smilesName='SMILES', molColName='Molecule', includeFingerprints=False)

# List of selected descriptors
selected_columns = [
    # Your list of selected columns
]

# Generate Morgan fingerprints
morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=6, fpSize=2048)  
def calculate_fcfp6(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fcfp6 = morgan_gen.GetFingerprint(mol)
    array = np.zeros((2048,), dtype=int)
    DataStructs.ConvertToNumpyArray(fcfp6, array)
    return array

# Generate Morgan fingerprints and convert to DataFrame
fcfp6_data = df['SMILES'].apply(calculate_fcfp6)
fcfp6_df = pd.DataFrame(fcfp6_data.tolist(), index=df.index)

# Combine descriptors with fingerprints
features = df[selected_columns].join(fcfp6_df).dropna()

# Ensure all column names are strings
features.columns = features.columns.astype(str)

# Extract target variable 'Stable'
target_column = 'Stable'
labels = df[target_column].loc[features.index].astype(int)

# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Apply PCA
pca = PCA(n_components=9)  # Adjust components as needed
principal_components = pca.fit_transform(scaled_features)

# Combine PCA transformed descriptors with Morgan fingerprints
combined_features = np.hstack((principal_components, fcfp6_df.loc[features.index].values))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    combined_features, labels, test_size=0.2, random_state=42
)

# Load the best Random Forest model (not calibrated)
best_rf_model = joblib.load('best_random_forest_model.pkl')

def compute_feature_importance(features, labels, pca_loadings, morgan_start_index):
    # Use the fitted Random Forest model
    rf_model = best_rf_model 
    importances = rf_model.feature_importances_

    # Create a DataFrame for feature importance contributions
    morgan_df_columns = [f'Morgan_{i}' for i in range(2048)]  # Define Morgan columns
    feature_names = selected_columns + ["Total Morgan Fingerprints"]  # Combine all feature names

    feature_importance_contributions = pd.DataFrame({
        'Feature': feature_names,
        'Importance': np.zeros(len(feature_names))
    })

    # Calculate contributions from PCA components
    num_components = min(len(importances), pca_loadings.shape[1])  # Limit iterations based on PCA loadings
    for i in range(num_components):  # Only PCA components
        component_importance = importances[i]
        for j in range(len(selected_columns)):  # Loop through descriptors
            if j < pca_loadings.shape[0]:  # Ensure j is within bounds for PCA loadings
                feature_importance_contributions.loc[feature_importance_contributions['Feature'] == selected_columns[j], 'Importance'] += (
                    np.abs(pca_loadings[j, i]) * component_importance
                )

    # Add combined Morgan fingerprint contribution
    total_morgan_importance = np.sum(importances[morgan_start_index:morgan_start_index + len(morgan_df_columns)])
    feature_importance_contributions.loc[feature_importance_contributions['Feature'] == "Total Morgan Fingerprints", 'Importance'] += total_morgan_importance

    # Sort and filter only relevant values
    feature_importance_contributions.sort_values(by='Importance', ascending=False, inplace=True)

    # Extract total Morgan importance and the top 10 descriptors
    return total_morgan_importance, feature_importance_contributions[['Feature', 'Importance']].head(10)

# Call feature importance analysis
morgan_start_index = len(selected_columns)  # Index for where Morgan fingerprints start
total_morgan_importance, top_importance_df = compute_feature_importance(X_train, y_train, pca.components_, morgan_start_index)

# Print total Morgan importance and top importance values
print(f"Total Morgan Fingerprint Importance: {total_morgan_importance:.6f}")
print("Top 10 Descriptor Importances:") 
print(top_importance_df)


Total Morgan Fingerprint Importance: 0.999989
Top 10 Descriptor Importances:
                     Feature  Importance
0  Total Morgan Fingerprints    0.999989
